## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-29-02-17-48 +0000,nyt,Jimmy Lai’s Freedom May Now Hinge on Beijing a...,https://www.nytimes.com/2025/08/27/world/asia/...
1,2025-08-29-02-16-31 +0000,nypost,RFK Jr’s deputy expected to replace ousted CDC...,https://nypost.com/2025/08/28/us-news/rfk-jrs-...
2,2025-08-29-02-11-21 +0000,nyt,Lawyers for Abrego Garcia Seek Gag Order Again...,https://www.nytimes.com/2025/08/28/us/politics...
3,2025-08-29-01-57-00 +0000,latimes,These California eggs are linked to a salmonel...,https://www.latimes.com/california/story/2025-...
4,2025-08-29-01-54-59 +0000,nyt,RFK Jr. Sought to Fire CDC Director Susan Mona...,https://www.nytimes.com/2025/08/28/health/rfk-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2368/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
6,trump,44
76,shooting,15
74,minneapolis,14
109,school,13
46,new,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
93,2025-08-28-21-40-00 +0000,wsj,"Fed governor Lisa Cook sued President Trump, s...",https://www.wsj.com/economy/central-banking/li...,124
209,2025-08-28-13-46-00 +0000,wsj,"President Trump fired Robert Primus, a board m...",https://www.wsj.com/politics/policy/trump-fire...,97
200,2025-08-28-14-18-31 +0000,cbc,U.S. Fed governor Lisa Cook suing Trump admini...,https://www.cbc.ca/news/world/trump-lisa-cook-...,97
185,2025-08-28-15-14-52 +0000,bbc,Can Trump fire Fed Governor Lisa Cook and why ...,https://www.bbc.com/news/articles/cedvj2d5538o...,94
181,2025-08-28-15-24-39 +0000,bbc,US Fed Governor Lisa Cook sues Trump over his ...,https://www.bbc.com/news/articles/c1dxl6ry4y3o...,93


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
93,124,2025-08-28-21-40-00 +0000,wsj,"Fed governor Lisa Cook sued President Trump, s...",https://www.wsj.com/economy/central-banking/li...
288,73,2025-08-28-03-19-08 +0000,nypost,Russian ballistic missiles hit Kyiv in large-s...,https://nypost.com/2025/08/27/world-news/russi...
64,60,2025-08-28-22-55-33 +0000,nypost,Trans Minneapolis school shooter Robin Westman...,https://nypost.com/2025/08/28/us-news/minnesot...
209,44,2025-08-28-13-46-00 +0000,wsj,"President Trump fired Robert Primus, a board m...",https://www.wsj.com/politics/policy/trump-fire...
62,36,2025-08-28-23-21-06 +0000,nyt,Annunciation Catholic Church Reels From Deadly...,https://www.nytimes.com/2025/08/28/us/catholic...
30,36,2025-08-29-00-41-41 +0000,nypost,One of 37 intel pros whose clearance was pulle...,https://nypost.com/2025/08/28/us-news/one-of-3...
88,35,2025-08-28-21-44-00 +0000,wsj,Senate Republicans expressed alarm at the abru...,https://www.wsj.com/politics/policy/cdc-direct...
42,33,2025-08-29-00-28-26 +0000,nypost,Vance takes swipe at Mamdani — says Ugandan-bo...,https://nypost.com/2025/08/28/us-news/vance-sa...
1,32,2025-08-29-02-16-31 +0000,nypost,RFK Jr’s deputy expected to replace ousted CDC...,https://nypost.com/2025/08/28/us-news/rfk-jrs-...
22,31,2025-08-29-00-53-40 +0000,missionlocal,S.F. immigration judge ordered a woman into co...,https://missionlocal.org/2025/08/six-arrested-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
